In [ ]:
!pip install streamlit --quiet
!pip install plotly.express
!npm install localtunnel --quiet
!pip install duckdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴Requirement already satisfied: duckdb in /usr/local/lib/python3.11/dist-packages (1.1.3)


In [ ]:
%%writefile Indicadores_Economicos.py

import random
import pandas as pd
import streamlit as st
import plotly.express as px
import duckdb
import plotly.graph_objects as go

st.set_page_config(page_title="Indicadores Economicos do Brasil Dashboard", page_icon=":bar_chart:", layout="wide")

st.title("Streamlit -- Dashboard - Indicadores Economicos do Brasil")
st.markdown("Esse dashborad visa facilitar a visualização de indicadores Economicos Brasileiros no periodo de 2002 à 2024")
st.markdown("---")

# Menu (lateral) - Upload de Arquivo
with st.sidebar:
    st.header("Adicione seu Arquivo")
    uploaded_file = st.file_uploader("Escolha o arquivo Excel")

if uploaded_file is None:
    st.info("Carregue um arquivo via lateral esquerda na opção 'Browse files'", icon="ℹ️")
    st.stop()

# Carregar os dados
@st.cache_data
def load_data(path):
    df = pd.read_excel(path, sheet_name="Séries")
    df.columns = [col.strip().upper() for col in df.columns]  # Padronizar nomes
    return df

df = load_data(uploaded_file)

# Converter a coluna de DATA para formato datetime
df["DATA"] = pd.to_datetime(df["DATA"].astype(str), format="%Y.%m")
df["DATA_DATE"] = df["DATA"].dt.date  # Converter para formato de data (sem horário)

# Filtro de período
st.sidebar.subheader("Filtro de Período")
min_date, max_date = df["DATA_DATE"].min(), df["DATA_DATE"].max()
start_date, end_date = st.sidebar.slider("Selecione o período", min_value=min_date, max_value=max_date, value=(min_date, max_date))

# Filtrar dados usando DuckDB
df_filtered = duckdb.sql(f"""
    SELECT * FROM df
    WHERE DATA_DATE BETWEEN '{start_date}' AND '{end_date}'
    ORDER BY DATA_DATE
""").df()


# Exibir dados brutos
with st.expander("Dados Brutos"):
    st.dataframe(df_filtered)
st.markdown("---")


# Gráfico de Linha para TAXA_SELIC, DLSP e IPCA
with st.expander("Evolução da TAXA SELIC, DLSP e IPCA"):
    query1 = duckdb.sql("""
        SELECT DATA_DATE, TAXA_SELIC, DLSP, IPCA FROM df_filtered
        ORDER BY DATA_DATE
    """).df()
    fig1 = px.line(query1, x="DATA_DATE", y=["TAXA_SELIC", "DLSP", "IPCA"], title="Evolução da TAXA SELIC, DLSP e IPCA")
    st.plotly_chart(fig1, use_container_width=True)
st.markdown("---")


# Gráfico de Linha para PIB_MILHOES
with st.expander("Evolução do PIB (em milhões)"):
    query2 = duckdb.sql("""
        SELECT DATA_DATE, PIB_MILHOES FROM df_filtered
        ORDER BY DATA_DATE
    """).df()
    fig2 = px.line(query2, x="DATA_DATE", y="PIB_MILHOES", title="Evolução do PIB (em milhões)")
    st.plotly_chart(fig2, use_container_width=True)
st.markdown("---")


with st.expander("Soma do PIB Anual"):
    # Agrupar por ano e somar o PIB
    df_filtered["ANO"] = df_filtered["DATA_DATE"].dt.year
    df_annual_pib = df_filtered.groupby("ANO")["PIB_MILHOES"].sum().reset_index()

    # Gráfico de barras
    fig3 = go.Figure(data=[go.Bar(x=df_annual_pib["ANO"], y=df_annual_pib["PIB_MILHOES"], marker_color='indianred')])
    fig3.update_layout(
        title="Soma do PIB Anual",
        xaxis_title="Ano",
        yaxis_title="PIB (em milhões)",
        template="plotly_dark"
    )
    st.plotly_chart(fig3, use_container_width=True)
st.markdown("---")


with st.expander("Média das Métricas por Ano"):
    # Agrupar por ano e calcular a média das métricas
    df_filtered["ANO"] = df_filtered["DATA_DATE"].dt.year
    df_avg_metrics = df_filtered.groupby("ANO")[["TAXA_SELIC", "DLSP", "IPCA"]].mean().reset_index()

    # Criar o gráfico de barras para cada métrica
    fig4 = go.Figure()

    # Adicionar as barras para cada métrica
    fig4.add_trace(go.Bar(x=df_avg_metrics["ANO"], y=df_avg_metrics["TAXA_SELIC"], name="Taxa Selic", marker_color='royalblue'))
    fig4.add_trace(go.Bar(x=df_avg_metrics["ANO"], y=df_avg_metrics["DLSP"], name="DLSP", marker_color='green'))
    fig4.add_trace(go.Bar(x=df_avg_metrics["ANO"], y=df_avg_metrics["IPCA"], name="IPCA", marker_color='orange'))

    fig4.update_layout(
        title="Média das Métricas por Ano",
        xaxis_title="Ano",
        yaxis_title="Valor Médio",
        barmode='group',  # Para exibir as barras lado a lado
        template="plotly_dark"
    )
    st.plotly_chart(fig4, use_container_width=True)
st.markdown("---")

Writing Indicadores_Economicos.py


In [ ]:
!curl ipv4.icanhazip.com

35.226.254.120


In [ ]:
!streamlit run Indicadores_Economicos.py & npx localtunnel --port 8501

⠙your url is: https://true-hoops-help.loca.lt



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.226.254.120:8501

  Stopping...
^C
